# 投資信託の評価

In [3]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import yfinance as yf

In [ ]:
# データの読み込み
path = '../../../../data/raw/investment_trust.parquet'
df = pl.read_parquet(path)
df.head()

isin_code,investment_trust_assosiation_japan_code,investment_name,date,nav,net_assets_million
str,str,str,date,i64,f64
null,"""0431U169""","""ifree_emerging_markets_bond_in…",2016-09-08,10053,1.005303
null,"""0431U169""","""ifree_emerging_markets_bond_in…",2016-09-09,10089,1.008859
null,"""0431U169""","""ifree_emerging_markets_bond_in…",2016-09-12,9994,1.936575
null,"""0431U169""","""ifree_emerging_markets_bond_in…",2016-09-13,9893,2.309483
null,"""0431U169""","""ifree_emerging_markets_bond_in…",2016-09-14,9912,2.98453


In [ ]:
# 銘柄名，コードの確認
df[['investment_trust_association_japan_code', 'investment_name']].unique()

investment_trust_assosiation_japan_code,investment_name
str,str
"""3312184""","""emaxis_slim_all_country_3_regi…"
"""0331A172""","""emaxis_slim_developed_governme…"
"""3312175""","""emaxis_slim_balance_8_asset_eq…"
"""0431U169""","""ifree_emerging_markets_bond_in…"
"""89311216""","""sbi_v_us_equity_index_fund"""
"""8931117C""","""sbi_emerging_markets_equity_in…"
"""0331C177""","""emaxis_slim_emerging_markets_e…"
"""0331418A""","""emaxis_slim_all_world_equity_a…"
"""3311187""","""emaxis_slim_us_equity_sp500"""


## オルカン

In [12]:
all_country = df.filter(pl.col('investment_trust_assosiation_japan_code') == '0331418A')
all_country.head()

isin_code,investment_trust_assosiation_japan_code,investment_name,date,nav,net_assets_million
str,str,str,date,i64,f64
"""JP90C000H1T1""","""0331418A""","""emaxis_slim_all_world_equity_a…",2018-10-31,10000,10.0
"""JP90C000H1T1""","""0331418A""","""emaxis_slim_all_world_equity_a…",2018-11-01,10077,10.0
"""JP90C000H1T1""","""0331418A""","""emaxis_slim_all_world_equity_a…",2018-11-02,10209,11.0
"""JP90C000H1T1""","""0331418A""","""emaxis_slim_all_world_equity_a…",2018-11-05,10228,35.0
"""JP90C000H1T1""","""0331418A""","""emaxis_slim_all_world_equity_a…",2018-11-06,10259,86.0
